## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2022/seasontype/2

In [1]:
SET_WEEK = 14
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_14


In [2]:
from scrapers.scrape_2022 import get_2022_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
logit = '../modeling/logit.pkl'
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(logit, 'rb') as file:
    LOGIT = pickle.load(file)

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2022 data

In [4]:
%%time

X = get_2022_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game',
             'offense_downs_Fourth Downs_PCT',
             'offense_passing_AVG',
             'offense_passing_YDS/G',
             'offense_passing_RTG',
             'offense_receiving_AVG',
             'defense_points_per_game',
             'defense_passing_AVG',
             'defense_receiving_AVG',
             'defense_rushing_YDS/G',
             'defense_passing_SYL_per_game',
             'offense_downs_Third Downs_ATT_per_game',
             'defense_downs_Third Downs_ATT_per_game',
             'defense_downs_First Downs_penalty_ratio',
             'offense_passing_TD_per_game',
             'defense_rushing_TD_per_game',
             'defense_pass_TD_per_rush_TD',
             'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 31.5 s


,Team,games_played,season,offense_points_per_game,offense_downs_Fourth Downs_PCT,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,defense_rushing_TD_per_game,defense_pass_TD_per_rush_TD,offense_pass_TD_to_INT
0,Kansas City Chiefs,13,2022,29.5,80.0,8.1,309.2,102.8,12.4,22.9,...,10.5,106.2,22.923077,11.692308,13.076923,0.058140,2.538462,0.538462,4.000000,3.000000
1,Buffalo Bills,13,2022,27.2,50.0,7.5,263.4,94.5,11.8,17.0,...,10.3,99.8,19.000000,12.000000,13.000000,0.091603,2.000000,0.692308,1.666667,2.363636


In [5]:
print(X.shape)

(32, 21)


In [6]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [7]:
X['games_played'].unique().tolist()

[13]

In [8]:
# X[X['games_played'] == 8]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_logit'] = LOGIT.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_logit'] + predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 5
predictions['pred_opt_combo'] = (predictions['pred_logit'] + 7*predictions['pred_catboost']) / 8

predictions.sort_values('pred_opt_combo', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Kansas City Chiefs,2022,week_14,0.450237,0.349410,0.234540,0.330388,0.414575,0.355830,0.419033
1,Buffalo Bills,2022,week_14,0.510619,0.111828,0.077250,0.185236,0.364678,0.249922,0.382921
2,Philadelphia Eagles,2022,week_14,0.510899,0.265670,0.169086,0.278238,0.360343,0.316847,0.379162


In [10]:
predictions.to_csv('weekly_runs_2022/' + week_name + '.csv', index = False)

### Check weekly run

In [11]:
predictions

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Kansas City Chiefs,2022,week_14,0.450237,0.349410,0.234540,0.330388,0.414575,0.355830,0.419033
1,Buffalo Bills,2022,week_14,0.510619,0.111828,0.077250,0.185236,0.364678,0.249922,0.382921
2,Philadelphia Eagles,2022,week_14,0.510899,0.265670,0.169086,0.278238,0.360343,0.316847,0.379162
8,San Francisco 49ers,2022,week_14,0.291603,0.245168,0.132817,0.280411,0.356967,0.261393,0.348796
6,Miami Dolphins,2022,week_14,0.194835,0.270114,0.141153,0.274884,0.366489,0.249495,0.345032
9,Dallas Cowboys,2022,week_14,0.243835,0.085872,0.071486,0.194954,0.346148,0.188459,0.333359
4,Cincinnati Bengals,2022,week_14,0.049889,0.141814,0.062455,0.208395,0.353837,0.163278,0.315844
12,Seattle Seahawks,2022,week_14,0.010875,0.100635,0.058458,0.210239,0.351696,0.146381,0.309093
17,Tampa Bay Buccaneers,2022,week_14,0.005294,0.055908,0.050640,0.180514,0.339360,0.126343,0.297602
3,Detroit Lions,2022,week_14,0.004076,0.074011,0.065280,0.213292,0.337496,0.138831,0.295819
